<h1>this code is run in my arudino</h1>

<h4>#define RELAY_HAPPY 2  // Relay pin for "happy" emotion mist maker
#define RELAY_SAD 3    // Relay pin for "sad" emotion mist maker

String inputString = "";  // Buffer to store serial input

void setup() {
  // Set relay pins as output
  pinMode(RELAY_HAPPY, OUTPUT);
  pinMode(RELAY_SAD, OUTPUT);

  // Ensure both mist makers are OFF at start
  digitalWrite(RELAY_HAPPY, HIGH);
  digitalWrite(RELAY_SAD, HIGH);

  Serial.begin(9600);  // Start serial communication
}

void loop() {
  // Check for incoming serial data
  while (Serial.available()) {
    char c = Serial.read();

    // Check for end of command
    if (c == '\n') {
      inputString.trim();  // Clean up the received string

      // Handle emotion commands
      if (inputString == "happy") {
        digitalWrite(RELAY_HAPPY, HIGH);  // Turn ON happy mist
        digitalWrite(RELAY_SAD, LOW);     // Turn OFF sad mist
      } else if (inputString == "sad") {
        digitalWrite(RELAY_HAPPY, LOW);
        digitalWrite(RELAY_SAD, HIGH);  // Turn ON sad mist
      } else if (inputString == "none") {
        // Any other input (like "none") turns both OFF
        digitalWrite(RELAY_HAPPY, HIGH);
        digitalWrite(RELAY_SAD, HIGH);
      }

      inputString = "";  // Clear the input buffer for next command
    } else {
      inputString += c;  // Build the input string one character at a time
    }
  }
}</h4>

<h1>universal</h1>

<h4>#define RELAY_HAPPY 2  // Relay pin for "happy" emotion mist maker
#define RELAY_SAD 3    // Relay pin for "sad" emotion mist maker

String inputString = "";  // Buffer to store serial input

void setup() {
  // Set relay pins as output
  pinMode(RELAY_HAPPY, OUTPUT);
  pinMode(RELAY_SAD, OUTPUT);

  // Ensure both mist makers are OFF at start
  digitalWrite(RELAY_HAPPY, LOW);
  digitalWrite(RELAY_SAD, LOw);

  Serial.begin(9600);  // Start serial communication
}

void loop() {
  // Check for incoming serial data
  while (Serial.available()) {
    char c = Serial.read();

    // Check for end of command
    if (c == '\n') {
      inputString.trim();  // Clean up the received string

      // Handle emotion commands
      if (inputString == "happy") {
        digitalWrite(RELAY_HAPPY, HIGH);  // Turn ON happy mist
        digitalWrite(RELAY_SAD, LOW);     // Turn OFF sad mist
      } else if (inputString == "sad") {
        digitalWrite(RELAY_HAPPY, LOW);
        digitalWrite(RELAY_SAD, HIGH);  // Turn ON sad mist
      } else if (inputString == "none") {
        // Any other input (like "none") turns both OFF
        digitalWrite(RELAY_HAPPY, LOW);
        digitalWrite(RELAY_SAD, LOw);
      }

      inputString = "";  // Clear the input buffer for next command
    } else {
      inputString += c;  // Build the input string one character at a time
    }
  }
}</h4>

<h1>Face Detection System</h1>

In [ ]:
import cv2
import numpy as np
import pygame
import serial
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load custom model
model = load_model(r"D:\ad\emotion_model.h5")

# Emotion labels based on training
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Connect to Arduino
arduino = serial.Serial('COM7', 9600)
time.sleep(2)

# Initialize pygame
pygame.mixer.init()

# Load face detector
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

# Music only for happy and sad
music_tracks = {
    "happy": r"D:\ad\music\happy.mp3",
    "sad": r"D:\ad\music\sad.mp3",
}

current_emotion = None

def apply_gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def enhance_frame(frame):
    frame = apply_gamma_correction(frame)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def play_music_and_trigger_relay(emotion):
    global current_emotion
    if emotion in ['happy', 'sad']:
        if current_emotion != emotion:
            current_emotion = emotion
            pygame.mixer.music.stop()
            pygame.mixer.music.load(music_tracks[emotion])
            pygame.mixer.music.play()
            arduino.write((emotion + "\n").encode())
    else:
        if current_emotion is not None:
            pygame.mixer.music.stop()
            arduino.write(b"none\n")  # This sends "none" to turn both mist makers off
            current_emotion = None


def analyze_emotion(frame, face_box):
    x, y, w, h = face_box
    face = frame[y:y+h, x:x+w]
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (48, 48))
    face = face.astype("float") / 255.0
    face = img_to_array(face)
    face = np.expand_dims(face, axis=0)
    preds = model.predict(face, verbose=0)
    emotion_idx = np.argmax(preds)
    return emotion_labels[emotion_idx]

frame_count = 0
emotion = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = enhance_frame(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        if current_emotion is not None:
            pygame.mixer.music.stop()
            arduino.write(b"none\n")
            current_emotion = None
        emotion = None

    for (x, y, w, h) in faces:
        # Analyze emotion every 5 frames
        if frame_count % 5 == 0:
            detected_emotion = analyze_emotion(frame, (x, y, w, h))
            emotion = detected_emotion
            play_music_and_trigger_relay(detected_emotion)

        # Create blurred background with clear face
        blurred_frame = cv2.GaussianBlur(frame, (25, 25), 30)
        face_roi = frame[y:y+h, x:x+w]
        blurred_frame[y:y+h, x:x+w] = face_roi
        frame = blurred_frame

        # Draw bounding box and emotion label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            emotion if emotion else "No Emotion",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2
        )

    cv2.imshow('Emotion-based Mist Maker Control', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
arduino.close()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
